# 🔬 State Preparation - CODE LAB

**Section 1.3: Quantum Operations**

> 📖 For concepts, see README_REORGANIZED.md

---

## Quick API Reference

| Operation | Syntax | Purpose |
|-----------|--------|---------|
| `initialize()` | `qc.initialize([a,b], qubit)` | Set arbitrary state (gate synthesis) |
| `reset()` | `qc.reset(qubit)` | Force to \|0⟩ (measurement-based) |
| `barrier()` | `qc.barrier()` | Visual separator, blocks optimization |
| `delay()` | `qc.delay(100, qubit, unit='ns')` | Timing control |

### ⚠️ Key Trap
```python
# These both give |0⟩ but work DIFFERENTLY:
qc.initialize([1, 0], 0)  # Adds gates (synthesis)
qc.reset(0)                # Uses measurement (faster)
```

## Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_state_city

def verify_state(qc, expected, name="State"):
    """Verify circuit produces expected state vector."""
    actual = Statevector(qc).data
    match = np.allclose(actual, expected, atol=1e-10)
    status = "✅ PASS" if match else "❌ FAIL"
    print(f"{name}: {status}")
    return match

print("✅ Setup complete")

---

## 1. initialize() - Arbitrary State Preparation

`qc.initialize([amplitude_0, amplitude_1, ...], qubits)`

In [ ]:
# initialize() - Set arbitrary quantum states
print("initialize() Examples")
print("=" * 50)

# Single qubit states
for name, amps in [('|1⟩', [0, 1]), ('|+⟩', [1/np.sqrt(2), 1/np.sqrt(2)]), ('|-⟩', [1/np.sqrt(2), -1/np.sqrt(2)])]:
    qc = QuantumCircuit(1)
    qc.initialize(amps, 0)
    print(f"{name}: {Statevector(qc).data}")

# Multi-qubit: Bell state |Φ+⟩ = (|00⟩+|11⟩)/√2
qc_bell = QuantumCircuit(2)
qc_bell.initialize([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], [0, 1])
print(f"\nBell |Φ+⟩: {Statevector(qc_bell).data}")

---

## 2. barrier() - Visual Separator

`qc.barrier()` - Doesn't change state, only affects optimization

In [ ]:
# barrier() - Visual separator (doesn't change state!)
qc = QuantumCircuit(3)
qc.h(range(3))
qc.barrier()
qc.cx(0, 1); qc.cx(1, 2)
qc.barrier()
qc.h(range(3))
display(qc.draw(output='mpl'))

# Prove barrier doesn't change state
qc1 = QuantumCircuit(1); qc1.h(0)
qc2 = QuantumCircuit(1); qc2.h(0); qc2.barrier()
print(f"barrier() changes state: {not np.allclose(Statevector(qc1).data, Statevector(qc2).data)}")
print("✅ barrier() is visual only - blocks optimization, not state")

---

## 3. reset() - Force to |0⟩

`qc.reset(qubit)` - Uses measurement, always returns |0⟩

In [ ]:
# reset() - Force qubit to |0⟩ (measurement-based)
qc = QuantumCircuit(2)
qc.x([0, 1])   # Both to |1⟩
qc.reset(0)    # Reset only q0
print(f"|11⟩ → reset(0) → {Statevector(qc).data}  (q0=|0⟩, q1=|1⟩)")
display(qc.draw(output='mpl'))

---

## 4. delay() - Timing Control

`qc.delay(time, qubit, unit='ns')` - For pulse-level experiments

In [ ]:
# delay() - Timing control (pulse-level)
qc = QuantumCircuit(1)
qc.h(0)
qc.delay(100, 0, unit='ns')  # Units: ns, us, ms
qc.x(0)
print("delay(time, qubit, unit='ns')")
display(qc.draw(output='mpl'))

---

## 5. ⚠️ Trap Demonstrations

In [ ]:
# ⚠️ TRAP: initialize() vs reset() - Both give |0⟩ differently!
print("TRAP: initialize([1,0]) vs reset()")
print("=" * 50)

qc_init = QuantumCircuit(1)
qc_init.x(0)  # Start |1⟩
qc_init.initialize([1, 0], 0)  # Gate synthesis

qc_reset = QuantumCircuit(1)
qc_reset.x(0)  # Start |1⟩
qc_reset.reset(0)  # Measurement-based

print(f"initialize: {Statevector(qc_init).data}")
print(f"reset:      {Statevector(qc_reset).data}")
print("\n✅ Same result, BUT:")
print("   initialize() = adds gates (synthesis)")
print("   reset() = measures + conditional X (faster!)")

### ⚠️ TRAP: initialize() is EXPENSIVE (adds many gates!)

The `initialize()` method synthesizes gates to create arbitrary states. For simple states, use gates directly!

In [ ]:
# TRAP: initialize() is EXPENSIVE - Compare gate counts!
from qiskit import transpile

print("⚠️ initialize() Gate Cost Analysis")
print("=" * 60)

# Method 1: initialize() for |+⟩ state
qc_init = QuantumCircuit(1)
qc_init.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)

# Method 2: H gate for |+⟩ state  
qc_h = QuantumCircuit(1)
qc_h.h(0)

# Decompose initialize to see actual gates
qc_init_decomposed = transpile(qc_init, basis_gates=['u', 'cx'])
qc_h_decomposed = transpile(qc_h, basis_gates=['u', 'cx'])

print("Creating |+⟩ state:")
print(f"  initialize(): {qc_init_decomposed.count_ops()} gates")
print(f"  h(0):         {qc_h_decomposed.count_ops()} gates")

# Multi-qubit is worse!
print("\n" + "=" * 60)
print("Creating Bell state |Φ+⟩:")

# Method 1: initialize() for Bell state
qc_init_bell = QuantumCircuit(2)
qc_init_bell.initialize([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], [0, 1])

# Method 2: H + CX for Bell state
qc_hcx = QuantumCircuit(2)
qc_hcx.h(0)
qc_hcx.cx(0, 1)

qc_init_bell_decomp = transpile(qc_init_bell, basis_gates=['u', 'cx'])
qc_hcx_decomp = transpile(qc_hcx, basis_gates=['u', 'cx'])

print(f"  initialize(): {qc_init_bell_decomp.count_ops()} gates")
print(f"  h(0) + cx():  {qc_hcx_decomp.count_ops()} gates")

print("\n📋 RULE: Use initialize() only for ARBITRARY states!")
print("   For standard states (|+⟩, Bell, GHZ), use gates directly.")
print("   initialize() synthesizes a complex gate decomposition.")

---

## 6. Common Patterns

In [ ]:
# Common Pattern: Multiple ways to create |+⟩
print("3 Ways to Create |+⟩")
for name, f in [('H gate', lambda: (QuantumCircuit(1), 'h', 0)),
                ('initialize', lambda: (QuantumCircuit(1), 'initialize', [1/np.sqrt(2), 1/np.sqrt(2)], 0)),
                ('RY(π/2)', lambda: (QuantumCircuit(1), 'ry', np.pi/2, 0))]:
    qc = QuantumCircuit(1)
    if name == 'H gate': qc.h(0)
    elif name == 'initialize': qc.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)
    else: qc.ry(np.pi/2, 0)
    print(f"{name:12}: {Statevector(qc).data}")

In [ ]:
# Pattern: H⊗n creates superposition of all 2^n states
print("Multi-qubit Superposition: H⊗n")
for n in [2, 3, 4]:
    qc = QuantumCircuit(n)
    for i in range(n): qc.h(i)
    amp = 1/np.sqrt(2**n)
    print(f"n={n}: {2**n} states, amplitude = {amp:.4f}")

---

## 7. 🎓 Code Challenges

In [ ]:
# CHALLENGES
print("Challenge 1: Create |-⟩ = (|0⟩-|1⟩)/√2 three ways")
for name, gates in [('H+Z', ['h', 'z']), ('X+H', ['x', 'h']), ('initialize', None)]:
    qc = QuantumCircuit(1)
    if gates:
        for g in gates: getattr(qc, g)(0)
    else:
        qc.initialize([1/np.sqrt(2), -1/np.sqrt(2)], 0)
    print(f"{name}: {np.round(Statevector(qc).data, 3)}")

# Challenge 2: VQE ansatz pattern
print("\nChallenge 2: VQE Ansatz")
theta = np.pi/4
qc = QuantumCircuit(2)
qc.ry(theta, [0,1]); qc.barrier()
qc.cx(0, 1); qc.barrier()
qc.ry(theta, [0,1])
display(qc.draw(output='mpl'))

---

> 📖 See README.md for concepts, analogies, mnemonics, and practice questions